# How People Decide what they want to do

** DEPRICATED ** This approach is going to the 

Directed graph approach

People (`pops`) have desires for things like wealth, science and industry. 

## Action
An anonymous function will choose a pop and run the following actions:
* decide which actions, if any, that pop is capable of
* if there is at least one desire with met requirements
    * the pop will take the action with the greatest desire
* if there isn't at least one action
    * desire for one of those items will increase



**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. Nodes that do this are commented out.



In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [5]:
local_user = "Billmanserver"
# Requires special client as 'username' is often unavailable. 
c = db.CosmosdbClient()

c.run_query(f"g.V().hasLabel('pop').has('username','{local_user}').valueMap()")
pops = c.clean_nodes(c.res)
pops[0]

{'name': 'Elpuerhou Lon',
 'objid': '4446619274817',
 'conformity': 0.393,
 'literacy': 0.603,
 'aggression': 0.649,
 'constitution': 0.305,
 'health': 0.5,
 'isInFaction': '7438213295991',
 'industry': 0.477,
 'wealth': 0.54,
 'factionLoyalty': 0.613,
 'isIdle': 'True',
 'username': 'Billmanserver',
 'objtype': 'pop',
 'id': '4446619274817'}

In [6]:
# mapping to the modules that make the app
sys.path.insert(0, "../../app")
import creators.maths as maths



Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

Casting and uploading the objectives

In [14]:
# # Drop the items, if they exist. 
c.run_query("g.V().hasLabel('action').has('username','notebook').drop()")
actions_yaml = yaml.safe_load(open("actions_pop.yaml"))['actions']
for i,o in enumerate(actions_yaml):
    actions_yaml[i]['objid'] = maths.uuid()
data = {"nodes":actions_yaml,'edges':[]}


# # Then Create the nodes and add them to the DB
c.upload_data('notebook',data)

# After creating the nodes, pulling them into the notebook for reference
# I won't re-create them here, unless I've updated the file. 
res = c.run_query("g.V().hasLabel('action').valueMap()")
actions = c.clean_nodes(c.res)
pd.DataFrame(actions)


,type,applies_to,effort,requires_attr,requirement_description,augments_self_properties,comment,objid,username,objtype,id,leadingdesire
0,patriot_education,pop,10,factionLoyalty;.1,population loyalty to faction must be > .1,"factionLoyalty,literacy,aggression;0.05,0.01,0.05",expand public education programs with a collec...,0791269160932,notebook,action,0791269160932,NaN
1,safety_nets,pop,15,wealth;.1,population wealth must be > .1,"constitution,wealth;0.05,-.05",increase the general health of the population,1810605803915,notebook,action,1810605803915,NaN
2,individual_education,pop,10,wealth;.1,population wealth must be > .1,"conformity,literacy;-0.05,0.1",increase the general health of the population,3517275702068,notebook,action,3517275702068,NaN
3,patriot_propoganda,pop,10,NaN,NaN,"factionLoyalty,conformity,aggression;.1,0.1,.1","build national pride, increasin faction loyalt...",2925323841613,notebook,action,2925323841613,NaN
4,build_infrastructure,pop,20,wealth;.5,population wealth must be > .5,"wealth,industrial_infrastructure;-0.05,1",increase industry by investing in local infras...,5320716355810,notebook,action,5320716355810,wealth
5,healthcare_initialtives,pop,15,wealth;.1,population wealth > .1,"health,wealth;0.01,-.01",increase the health and literacy of other popu...,2652885336228,notebook,action,2652885336228,NaN
6,individual_education,pop,15,wealth;.1,population wealth > .1,"wealth,literacy,factionLoyalty;-.1,0.01,-.01","expand privatized education, increase literacy...",7503240323392,notebook,action,7503240323392,NaN


Actions that have `applies_to:"pop"` will be loaded to new populations as a default. 